<a href="https://colab.research.google.com/github/katarinagresova/M7DataSP_2020/blob/main/assignment_02/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data TODOs:
- choose dataset
- set way of loading data to this notebook

# Preprocessing TODOs:
- check missing values
- do one-hot encoding
- (optional: more preprocessing)
- split label
- split train and test

# Model training TODOs:
- create normalizer
- define neural network
- train neural network
- plot the evolution of loss and/or accuracy as a number of epoch increases

#Evaluating TODOs
- evaluate model on test data